In [62]:
!pip install azure-identity
!pip install langchain
!pip install openai
!pip install azure-search-documents

In [63]:
import os
import openai
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

In [69]:

!pip3 install python-dotenv
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# If your enviornment variable giving you previous values,
# then delete existing one And know set new one
# del os.environ['AZURE_SEARCH_KEY']

# Replace these with your own values, either in environment variables or directly here
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT")  # This is only in use if you want to store your data in azure blob storage
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER")

AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX")
AZURE_SEARCH_KEY = os.environ.get("AZURE_SEARCH_KEY")

AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT")
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT")
AZURE_OPENAI_CHATGPT_MODEL = os.environ.get("AZURE_OPENAI_CHATGPT_MODEL")

OPENAI_API_KEY= os.environ.get("OPENAI_API_KEY")
# AZURE_TENANT_ID = os.environ.get("AZURE_TENANT_ID")
# AZURE_CLIENT_SECRET = os.environ.get["AZURE_CLIENT_SECRET"]
# AZURE_CLIENT_ID = os.environ.get["AZURE_CLIENT_ID"]

KB_FIELDS_CONTENT = os.environ.get("KB_FIELDS_CONTENT")
KB_FIELDS_CATEGORY = os.environ.get("KB_FIELDS_CATEGORY")
KB_FIELDS_SOURCEPAGE = os.environ.get("KB_FIELDS_SOURCEPAGE")


In [71]:
# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed,
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the
# keys for each service
azure_credential = DefaultAzureCredential()


# Used by the OpenAI SDK
openai.api_type = "azure"
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-05-15"


# Comment these two lines out if using keys, set your API key in the OPENAI_API_KEY environment variable instead
# openai.api_type = "azure_ad"
# openai.api_key = azure_credential.get_token("https://cognitiveservices.azure.com/.default").token
openai.api_key=OPENAI_API_KEY

# Set up clients for Cognitive Search and Storage
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=AZURE_SEARCH_KEY)


In [79]:
# Chat roles
SYSTEM = "system"
USER = "user"
ASSISTANT = "assistant"

# previous chat
system_message_chat_conversation = """Amazon Elastic Compute Cloud (Amazon EC2) is a web service provided by Amazon Web Services (AWS) that allows users to rent virtual servers,
known as instances, on-demand. These instances can be used to run applications, host websites, process data, and perform various computing tasks in the cloud. EC2 provides a
scalable and flexible infrastructure for computing resources without the need for upfront investments in hardware.
Key features of Amazon EC2 include:

1. **Elasticity:** EC2 instances can be easily scaled up or down based on demand. This enables users to add or remove instances as needed, ensuring optimal performance and cost efficiency.

2. **Variety of Instance Types:** EC2 offers a wide range of instance types optimized for different use cases, such as compute-intensive, memory-intensive, GPU-accelerated, and more. Users can choose instances with the right balance of CPU, memory, storage, and networking capabilities for their applications.

3. **Customization:** Users can configure the operating system, networking, security settings, and other parameters of their EC2 instances. This allows for a high degree of customization to meet specific application requirements.

4. **Pay-as-You-Go Pricing:** EC2 operates on a pay-as-you-go model, where users are charged based on the instance types they use and the duration of usage. This makes it cost-effective, as users only pay for the resources they actually consume.

5. **Security:** EC2 instances can be deployed within Amazon Virtual Private Cloud (VPC) environments, providing network isolation and control over security settings. Users can also employ security groups and access control lists to regulate inbound and outbound traffic.

6. **Data Storage:** EC2 instances can be paired with various Amazon storage services, such as Amazon Elastic Block Store (EBS) for durable block storage and Amazon S3 for object storage. This enables users to attach and manage storage volumes for their instances.

7. **Load Balancing and Auto Scaling:** EC2 instances can be integrated with other AWS services like Elastic Load Balancing (ELB) and Auto Scaling, allowing for automatic distribution of traffic across instances and dynamic scaling based on traffic fluctuations.

8. **Global Availability:** EC2 instances can be deployed in multiple AWS regions and Availability Zones worldwide, providing geographic redundancy and fault tolerance.

Amazon EC2 has become a foundational service in cloud computing, enabling businesses and developers to deploy applications and services quickly and easily without the need to manage physical hardware. It has been widely adopted across various industries due to its scalability, flexibility, and cost-effectiveness..
"""
chat_conversations = [{"role" : SYSTEM, "content" : system_message_chat_conversation}]

summary_prompt_template = """Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. Generate a search query based on the conversation and the new question. Source names are not good search terms to include in the search query.

Summary:
{summary}

Question:
{question}

Search query:
"""

In [80]:
# Execute this cell multiple times updating user_input to accumulate chat history
user_input = "What is amazon ec2 ?"

# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

# To check if there is any chat history available
if len(chat_conversations) > 1:
    # If there is previous chat then provide the summary
    query_completion = openai.Completion.create(
        engine=AZURE_OPENAI_GPT_DEPLOYMENT,
        prompt=summary_prompt_template.format(summary=str(chat_conversations), question=user_input),
        temperature=0.7,
        max_tokens=32,
        stop=["\n"])
    search = query_completion.choices[0].text
else:
  # set the user input to search if there is no chat history
    search = user_input


In [81]:
# Alternatively simply use search_client.search(q, top=3) if not using semantic search
print("Searching:", search)
print("-------------------")
filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None
r = search_client.search(search,
                         filter=filter,
                        #  query_type=QueryType.SEMANTIC,
                         query_language="en-us",
                         query_speller="lexicon",
                         semantic_configuration_name="default",
                         top=3)



Searching: What is amazon ec2 ?
-------------------


In [83]:
# Assuming KB_FIELDS_SOURCEPAGE and KB_FIELDS_CONTENT are the actual field names you're using
results = []

for doc in r:
    source_page = doc.get(KB_FIELDS_SOURCEPAGE, "")
    content = doc.get(KB_FIELDS_CONTENT, "").replace("\n", "").replace("\r", "")
    result = f"{source_page}: {content}"
    results.append(result)

content = "\n".join(results)
user_content = user_input + " \nSOURCES:\n" + content
print(user_content)

What is amazon ec2 ? 
SOURCES:



In [84]:
user_content = user_input + " \nSOURCES:\n" + content

chat_conversations.append({"role": USER, "content": user_content })

chat_completion = openai.ChatCompletion.create(
    deployment_id=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    model=AZURE_OPENAI_CHATGPT_MODEL,
    messages=chat_conversations,
    temperature=0.7,
    max_tokens=1024,
    n=1)
chat_content = chat_completion.choices[0].message.content
'''
reset user content to avoid sources in conversation history
add source as a single shot in query conversation
'''
chat_conversations[-1]["content"] = user_input
chat_conversations.append({"role":ASSISTANT, "content": chat_content})

print("\n-------------------\n")
[print(conversation) for conversation in chat_conversations]


-------------------

{'role': 'system', 'content': 'Amazon Elastic Compute Cloud (Amazon EC2) is a web service provided by Amazon Web Services (AWS) that allows users to rent virtual servers, \nknown as instances, on-demand. These instances can be used to run applications, host websites, process data, and perform various computing tasks in the cloud. EC2 provides a \nscalable and flexible infrastructure for computing resources without the need for upfront investments in hardware.\nKey features of Amazon EC2 include:\n\n1. **Elasticity:** EC2 instances can be easily scaled up or down based on demand. This enables users to add or remove instances as needed, ensuring optimal performance and cost efficiency.\n\n2. **Variety of Instance Types:** EC2 offers a wide range of instance types optimized for different use cases, such as compute-intensive, memory-intensive, GPU-accelerated, and more. Users can choose instances with the right balance of CPU, memory, storage, and networking capabilit

[None, None, None]